In [6]:
import pymysql, random

# 连接到MySQL数据库
conn = pymysql.connect(
    host="vultr-prod-0735f93d-5e03-4e4b-b1ae-878c590715ab-vultr-prod-7d22.vultrdb.com",     # MySQL服务器的主机名或IP地址
    user="vultradmin", # MySQL用户名
    password="AVNS_PcAfxyST1X5nxEzfYSq", # MySQL密码
    database="order",
    port=16751
)

# 创建一个游标对象，用于执行SQL查询
cursor = conn.cursor()


In [8]:
def get_all_data():
    # 從mysql中讀取data
    # prompt = '''SELECT * FROM master'''
    # cursor.execute(prompt)
    # result = cursor.fetchall()
    # for i in result:
    #     print(i)
    
    prompt = '''SELECT `COLUMN_NAME` 
FROM `INFORMATION_SCHEMA`.`COLUMNS` 
WHERE `TABLE_SCHEMA`='order' 
    AND `TABLE_NAME`='master';'''
    cursor.execute(prompt)
    for i in cursor.fetchall():
        print(i)

get_all_data()

('id',)
('status',)
('total',)


In [ ]:
def create_order(data):
    letters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    numbers = '0123456789'
    random_letters = ''.join(random.choice(letters) for _ in range(2))
    random_numbers = ''.join(random.choice(numbers) for _ in range(3))

    totals

    order_idx = '#'+random_letters+random_numbers
    status = '0'
    total = 100

    # 寫入mysql dataset
    
    prompt = f'''INSERT INTO master (item,flavor,qty) VALUES (%s,%s,%s)'''

    return order_idx